In [1]:
# cian parser 
# try on jupiter not colab 

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import numpy as np

In [ ]:
driver = webdriver.Chrome('C:\chromedriver') 

In [ ]:
def get_html(url, PARAMS = None): #give html code of url page
    html = driver.page_source
    return html

def get_content(html): #give BS html code
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('div', class_ = '_93444fe79c--container--2Kouc')
    return items

def get_urls(): #collect urls to flat from sotred pages 
    urls = []
    params = ['&sort=creation_date_desc', 
              '&sort=creation_date_asc', 
              '&sort=price_object_order', 
              '&sort=total_price_desc', 
              '&sort=street_name', 
              '&sort=walking_time']
    
    for par in tqdm(range(len(params))):
        param = params[par] 
        for i in range(1,55): 
            URL = driver.get(f'https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p={i}&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1{param}')
            html = get_html(URL)
            items = get_content(html)
            for item in items:
                urls.append(item.find('a', class_='_93444fe79c--link--39cNw').get('href'))
        urls = list(set(urls))
    return urls

In [ ]:
def get_data(urls): #crutch for parse all falt params 
    prices, metro, floor, max_floor, size, msize, deadline, type_, bathroom, height, repairs, view, house = [], [], [], [], [], [], [], [], [], [], [], [], []
    garbage_chute, elevator, entrance, rooms, time = [], [], [], [], []
    for i in tqdm(range(len(urls))):
        url = urls[i]
        URL = driver.get(url)
        html = get_html(URL)
        soup = BeautifulSoup(html, 'html.parser')
        
        try: 
            price = soup.find('span', class_ = 'a10a3f92e9--price_value--1iPpd').find('span').get('content')
            prices.append(int(price.replace(' ', '').rstrip('₽')))
        except AttributeError: prices.append(np.nan)
            
        try: metro.append(soup.find('a', class_ = 'a10a3f92e9--underground_link--AzxRC').get_text())
        except AttributeError: metro.append(np.nan)
           
        try: time.append(int(soup.find('span', class_ = 'a10a3f92e9--underground_time--1fKft').get_text()[4:6]))
        except AttributeError: time.append(np.nan)
        except ValueError: time.append(np.nan)
         
        try: rooms.append(int(soup.find('h1', class_ = 'a10a3f92e9--title--2Widg').get_text()[:1]))
        except ValueError: rooms.append(np.nan)
            
        block1 = soup.find_all('div', class_ = 'a10a3f92e9--info-value--18c8R')
        titles1 = soup.find_all('div', class_ = 'a10a3f92e9--info-title--2bXM9')
        
        flag = [0,0,0,0]
        for i in range(len(titles1)):
            if titles1[i].get_text() == 'Общая':
                flag[0] = 1
                try: size.append(float(block1[i].get_text().rstrip('\xa0м²').replace(',','.')))
                except ValueError: size.append(np.nan)
            elif titles1[i].get_text() == 'Жилая':
                flag[1] = 1
                try: msize.append(float(block1[i].get_text().rstrip('\xa0м²').replace(',','.')))
                except ValueError: msize.append(np.nan)
            elif titles1[i].get_text() == 'Этаж':
                flag[2] = 1
                floors = block1[i].get_text().split(' из ')
                floor.append(int(floors[0]))
                max_floor.append(int(floors[1]))
            elif titles1[i].get_text() == 'Построен' or titles1[i].get_text() == 'Срок сдачи':
                flag[3] = 1
                deadline.append(int(block1[i].get_text()[-4:]))
        if flag[0] == 0:
            size.append(np.nan)
        if flag[1] == 0:
             msize.append(np.nan)
        if flag[2] == 0:
            floor.append(np.nan)
            max_floor.append(np.nan)
        if flag[3] == 0:
            deadline.append(np.nan)

        block2 = soup.find_all('span', class_ = 'a10a3f92e9--value--3Ftu5')
        titles2 = soup.find_all('span', class_ = 'a10a3f92e9--name--3bt8k')

        flag = [0,0,0,0]
        for i in range(len(titles2)):
            if titles2[i].get_text() == 'Тип жилья':
                flag[0] = 1
                type_.append(block2[i].get_text())
            elif titles2[i].get_text() == 'Высота потолков':
                flag[1] = 1
                height.append(float(block2[i].get_text().rstrip('м').replace(',','.')))
            elif titles2[i].get_text() == 'Санузел':
                flag[2] = 1
                bathroom.append(block2[i].get_text())
            elif titles2[i].get_text() == 'Вид из окон':
                flag[3] = 1
                view.append(block2[i].get_text())
        if flag[0] == 0:
            type_.append(np.nan)
        if flag[1] == 0:
             height.append(np.nan)
        if flag[2] == 0:
            bathroom.append(np.nan)
        if flag[3] == 0:
            view.append(np.nan)
        
    return prices, metro, size, msize, floor, max_floor, deadline, type_, height, bathroom, view, rooms, time